# Camera calibration notebook

In [ ]:
import cv2
import numpy as np
import glob

In [ ]:
# The checkerboard in data/calibration_images is 7x10, so we define the size as
checker_size = (6,9)
# Define criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# 3D points for each checkerboard image
objpoints = []
# 2D points for each checkerboard image
imgpoints = [] 

# World coordinates for 3D points..
worldObjpoints = np.zeros((1, checker_size[0] * checker_size[1], 3), np.float32)
# worldObjpoints = [[0,0,0], ..., [5,0,0], [0,1,0], ..., [5,1,0], [0,2,0], ..., [5,8,0]]
worldObjpoints[0,:,:2] = np.mgrid[0:checker_size[0], 0:checker_size[1]].T.reshape(-1, 2)
prev_img_shape = None

Given the checkerboard size, i.e., expected number of corners, and the world coordinates of 3D points (as assumed above), we relate the `worldObjpoints` with the resulting pixel coordinates `corners` (below).

In [ ]:
# Extracting path of individual image stored in a given directory
images = glob.glob('../data/calibration_images/*.jpg')
for fname in images:
    # Read image and convert to grayscale
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # We use the OpenCV function that finds corners of the checkerboard given 'checker_size'
    found, corners = cv2.findChessboardCorners(gray, checker_size, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    # If desired number of corner are found..
    if (found):
        objpoints.append(worldObjpoints)
        # Refine the pixel coordinates using the cornerSubPix function
        refined_corners = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(refined_corners)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, checker_size, refined_corners, found)
    
    cv2.imshow('Calibration image', img)
    cv2.waitKey(0)
cv2.destroyAllWindows()

Using the `objpoints` and the corresponding `imgpoints`, we can find the mapping from one to the other using the `calibrateCamera` function, and the 

In [ ]:
# The calibrateCamera function of OpenCV needs the value of known 3D points (objpoints)
# and the corresponding detected pixel coordinates (imgpoints)
ret, camMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
# Print values
print("Camera matrix: \n")
print(camMatrix)
print("Distortion coefficients: \n")
print(distCoeffs)
print("rvecs: \n")
print(rvecs)
print("tvecs: \n")
print(tvecs)

Moreover, we can refine the derived camera parameters using `getOptimalNewCameraMatrix`.

In [ ]:
img = cv2.imread(images[0])
h,w = img.shape[:2]
# Refine the camera matrix using parameters obtained by calibration
newCamMatrix, roi = cv2.getOptimalNewCameraMatrix(camMatrix, distCoeffs, (w,h), 1, (w,h))

Using the camera parameters we can undistort an image.

In [ ]:
# Method 1 to undistort the image
undst1 = cv2.undistort(img, camMatrix, distCoeffs, None, newCamMatrix)

# Method 2 to undistort the image
mapx, mapy = cv2.initUndistortRectifyMap(camMatrix, distCoeffs, None, newCamMatrix, (w,h), 5)

undst2 = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

# Displaying the undistorted image
cv2.imshow("Undistorted image 1", undst1)
cv2.waitKey(0)
cv2.imshow("Undistorted image 2", undst2)
cv2.waitKey(0)

